In [58]:
import stim
import numpy as np
from color_code_stim.cultivation import _load_cultivation_circuit

circuit: stim.Circuit = _load_cultivation_circuit(5, 1e-3)

In [59]:
from ldpc_post_selection.bplsd_decoder import SoftOutputsBpLsdDecoder

decoder = SoftOutputsBpLsdDecoder(circuit=circuit)

In [60]:
sampler = circuit.compile_detector_sampler()
det, obs = sampler.sample(1000, separate_observables=True)

In [61]:
import numpy as np

fails = []
soft_outputs = []
for det_sng, obs_sng in zip(det, obs):
    corr, _, _, soft_outputs_sng = decoder.decode(det_sng)
    inferred_obs = (corr @ decoder.obs_matrix.T.astype("uint8")) % 2
    fail = np.any(inferred_obs != obs_sng)
    fails.append(fail)
    soft_outputs.append(soft_outputs_sng)

In [62]:
total_llr_sum = np.sum(decoder.bit_llrs)

metrics = []
for soft_outputs_sng in soft_outputs:
    cluster_llrs = soft_outputs_sng["cluster_llrs"]
    cluster_llr_2norm_frac = 1 - np.sqrt(np.sum(cluster_llrs[:-1] ** 2)) / total_llr_sum
    metrics.append(cluster_llr_2norm_frac)

In [63]:
metrics = np.array(metrics)

In [64]:
np.histogram(metrics)

(array([870,   0,   0,   0,   0,   0,   0,   0,   0, 130]),
 array([1.77672282e-04, 1.00159905e-01, 2.00142138e-01, 3.00124371e-01,
        4.00106603e-01, 5.00088836e-01, 6.00071069e-01, 7.00053302e-01,
        8.00035534e-01, 9.00017767e-01, 1.00000000e+00]))